* filename_queue
* tf.train.batch

In [2]:
import tensorflow as tf
tf.set_random_seed(777)  # for reproducibility

filename_queue = tf.train.string_input_producer(
    ['data-01-test-score.csv'], shuffle=False, name='filename_queue')

reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

# Default values, in case of empty columns. Also specifies the type of the
# decoded result.
# 데이터타입 정의한거 넘겨줌
record_defaults = [[0.], [0.], [0.], [0.]]
xy = tf.decode_csv(value, record_defaults=record_defaults)

# collect batches of csv in
# 노드 이름 정하고 x, y정의하고 한번 펌프할때마다 10개씩 가져와라
train_x_batch, train_y_batch = \
    tf.train.batch([xy[0:-1], xy[-1:]], batch_size=10)

# placeholders for a tensor that will be always fed.
# shape 신경쓰기
X = tf.placeholder(tf.float32, shape=[None, 3])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([3, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Hypothesis
hypothesis = tf.matmul(X, W) + b

# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

# Start populating the filename queue.
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

for step in range(2001):
    x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
    cost_val, hy_val, _ = sess.run(
        [cost, hypothesis, train], feed_dict={X: x_batch, Y: y_batch})
    if step % 100 == 0:
        print(step, "Cost: ", cost_val, "\nPrediction:\n", hy_val)

coord.request_stop()
coord.join(threads)

# Ask my score
# 결과는 numpy와 같음
print("Your score will be ",
      sess.run(hypothesis, feed_dict={X: [[100, 70, 101]]}))

print("Other scores will be ",
      sess.run(hypothesis, feed_dict={X: [[60, 70, 110], [90, 100, 80]]}))

0 Cost:  8413.924 
Prediction:
 [[235.22784]
 [282.40146]
 [278.39618]
 [303.91577]
 [214.62396]
 [159.15425]
 [235.22784]
 [282.40146]
 [278.39618]
 [303.91577]]
100 Cost:  3.985147 
Prediction:
 [[138.93044 ]
 [103.349815]
 [152.63045 ]
 [183.14574 ]
 [180.58832 ]
 [197.39503 ]
 [138.93044 ]
 [103.349815]
 [152.63045 ]
 [183.14574 ]]
200 Cost:  3.8296158 
Prediction:
 [[180.71048]
 [197.5217 ]
 [139.05542]
 [103.43671]
 [152.71745]
 [183.28987]
 [180.71048]
 [197.5217 ]
 [139.05542]
 [103.43671]]
300 Cost:  2.6588426 
Prediction:
 [[152.62097]
 [183.21205]
 [180.61469]
 [197.41092]
 [139.01086]
 [103.39827]
 [152.62097]
 [183.21205]
 [180.61469]
 [197.41092]]
400 Cost:  3.8730483 
Prediction:
 [[139.00648]
 [103.38931]
 [152.56944]
 [183.18723]
 [180.57166]
 [197.35756]
 [139.00648]
 [103.38931]
 [152.56944]
 [183.18723]]
500 Cost:  3.72995 
Prediction:
 [[180.69046 ]
 [197.48035 ]
 [139.1267  ]
 [103.471855]
 [152.65506 ]
 [183.32603 ]
 [180.69046 ]
 [197.48035 ]
 [139.1267  ]
 [103

In [ ]:
'''
shuffle_batch
min_after_dequeue = 1000
capacity = min_after_dequeue + 3 * batch_size
example_batch, label_batch = tf.train.shuffle_batch(
[example, label], batch_size = batch_size, capacity = capacity,
min_after_dequeue = min_after_dequeue)
'''